In [0]:
PATH = 'Brain-Explore/'

In [0]:
import numpy as np

import pandas as pd

from nilearn.input_data import NiftiMasker

from load_confounds import load_confounds

In [0]:
bold_file = 'sub-01_ses-001_task-wm_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

mask_file = 'sub-01_ses-001_task-wm_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'

cnf_file = 'sub-01_ses-001_task-wm_run-01_desc-confounds_regressors.tsv'

eprime_csv_file = 'p01_WM_run1.csv'

## Task Block time extraction

In [0]:
# Extract task cue and fixation blocks from the Procedure column 

def get_task_procs(run):

  procs = run['Procedure'].dropna()
  indices = np.logical_or.reduce([procs.str.contains(i) for i in ['Cue','Fix']])
  return procs[indices].reset_index(drop=True)

In [0]:
# Check that the order of the blocks are 2 cue tasks followed by 1 fixation  

def check_procs_order(procs):

  for i in np.arange(0,len(procs),3):
    assert 'Cue' in procs[i] and 'Cue' in procs[i+1], "CueNBackPROC block not found in expected order"
    assert procs[i+2] == 'Fix15secPROC', "Fix15secPROC block not found in expected order"

  print("The task blocks are in expected order as defined by the protocol")

In [0]:
run1 = pd.read_csv(PATH + eprime_csv_file)

In [18]:
procs = get_task_procs(run1)
check_procs_order(procs)

The task blocks are in expected order as defined by the protocol


In [0]:
# Extract the columns that indicate the block labels - task type and stimulus type

def get_block_labels(run):
  
  labels = run[['BlockType', 'StimType']].dropna().drop_duplicates().reset_index(drop=True)
  labels.columns = ['block_type','stim_type']
  return labels

In [0]:
# Return an iterator for the specified onset time attribute

def iterator(run, name):

  onset = '{}.OnsetTime'
  return iter(run[onset.format(name)].dropna().values)  

In [0]:
# Compute the start and end times of each of the task blocks based on the order

def get_block_times(run, blocks):

  cue_2b_onsets = iterator(run, 'Cue2Back')
  cue_0b_onsets = iterator(run, 'CueTarget') 
  fix_onsets = iterator(run, 'Fix15sec') 
  
  new_cols = ['start_time', 'end_time']
  blocks = blocks.reindex(columns = blocks.columns.tolist() + new_cols)

  for index, item in blocks.iterrows():
    blocks.at[index,'start_time'] = next(cue_2b_onsets) if item['block_type'] == '2-Back' else next(cue_0b_onsets)
    
  for index, item in blocks.iterrows():
    blocks.at[index,'end_time'] = blocks.at[index+1,'start_time'] if index % 2 == 0 else next(fix_onsets)

  return blocks

In [22]:
blocks = get_block_labels(run1)
blocks

,block_type,stim_type
0,2-Back,Body
1,0-Back,Face
2,2-Back,Tools
3,0-Back,Body
4,0-Back,Place
5,2-Back,Face
6,0-Back,Tools
7,2-Back,Place


In [23]:
blocks = get_block_times(run1, blocks)
blocks

,block_type,stim_type,start_time,end_time
0,2-Back,Body,42456.0,70321.0
1,0-Back,Face,70321.0,98151.0
2,2-Back,Tools,113152.0,140984.0
3,0-Back,Body,140984.0,168812.0
4,0-Back,Place,183815.0,211646.0
5,2-Back,Face,211646.0,239472.0
6,0-Back,Tools,254477.0,282309.0
7,2-Back,Place,282309.0,310132.0


In [0]:
# Calibrate the times to start from zero by subtracting with the sync onset time

def adjust_block_times(run, blocks):

  sync_start_time = run['SyncSlide.OnsetTime'].dropna().values[0]
  blocks['start_time'] -= sync_start_time
  blocks['end_time'] -= sync_start_time

  return blocks


In [25]:
adjust_block_times(run1, blocks)

,block_type,stim_type,start_time,end_time
0,2-Back,Body,8013.0,35878.0
1,0-Back,Face,35878.0,63708.0
2,2-Back,Tools,78709.0,106541.0
3,0-Back,Body,106541.0,134369.0
4,0-Back,Place,149372.0,177203.0
5,2-Back,Face,177203.0,205029.0
6,0-Back,Tools,220034.0,247866.0
7,2-Back,Place,247866.0,275689.0


In [0]:
# Compute block duration from start and end times

def compute_block_durations(blocks):

  blocks['duration'] = blocks['end_time'] - blocks['start_time']
  return blocks

In [27]:
blocks = compute_block_durations(blocks)
blocks

,block_type,stim_type,start_time,end_time,duration
0,2-Back,Body,8013.0,35878.0,27865.0
1,0-Back,Face,35878.0,63708.0,27830.0
2,2-Back,Tools,78709.0,106541.0,27832.0
3,0-Back,Body,106541.0,134369.0,27828.0
4,0-Back,Place,149372.0,177203.0,27831.0
5,2-Back,Face,177203.0,205029.0,27826.0
6,0-Back,Tools,220034.0,247866.0,27832.0
7,2-Back,Place,247866.0,275689.0,27823.0


In [28]:
TR = 1490
NUM_FRAMES = 202

frames = np.arange(NUM_FRAMES) * TR

frames

array([     0,   1490,   2980,   4470,   5960,   7450,   8940,  10430,
        11920,  13410,  14900,  16390,  17880,  19370,  20860,  22350,
        23840,  25330,  26820,  28310,  29800,  31290,  32780,  34270,
        35760,  37250,  38740,  40230,  41720,  43210,  44700,  46190,
        47680,  49170,  50660,  52150,  53640,  55130,  56620,  58110,
        59600,  61090,  62580,  64070,  65560,  67050,  68540,  70030,
        71520,  73010,  74500,  75990,  77480,  78970,  80460,  81950,
        83440,  84930,  86420,  87910,  89400,  90890,  92380,  93870,
        95360,  96850,  98340,  99830, 101320, 102810, 104300, 105790,
       107280, 108770, 110260, 111750, 113240, 114730, 116220, 117710,
       119200, 120690, 122180, 123670, 125160, 126650, 128140, 129630,
       131120, 132610, 134100, 135590, 137080, 138570, 140060, 141550,
       143040, 144530, 146020, 147510, 149000, 150490, 151980, 153470,
       154960, 156450, 157940, 159430, 160920, 162410, 163900, 165390,
      

In [0]:
# Compute the frmae indices of the time series based on block times and TR with adjustment(#TRs) for hemodynamic delay

def compute_frame_indices(blocks, TR, adjustment=0):

  blocks['start_index'] = np.ceil(blocks['start_time']/TR).astype(np.int64) + adjustment
  blocks['end_index'] = np.ceil(blocks['end_time']/TR).astype(np.int64) + adjustment
  blocks['num_frames'] = np.ceil(blocks['duration']/TR).astype(np.int64) + adjustment

  return blocks

In [31]:
blocks = compute_frame_indices(blocks, TR)
blocks

,block_type,stim_type,start_time,end_time,duration,start_index,end_index,num_frames
0,2-Back,Body,8013.0,35878.0,27865.0,6,25,19
1,0-Back,Face,35878.0,63708.0,27830.0,25,43,19
2,2-Back,Tools,78709.0,106541.0,27832.0,53,72,19
3,0-Back,Body,106541.0,134369.0,27828.0,72,91,19
4,0-Back,Place,149372.0,177203.0,27831.0,101,119,19
5,2-Back,Face,177203.0,205029.0,27826.0,119,138,19
6,0-Back,Tools,220034.0,247866.0,27832.0,148,167,19
7,2-Back,Place,247866.0,275689.0,27823.0,167,186,19


## Loading confounds in masker to obtain  BOLD data time series

In [37]:
confounds = load_confounds(PATH + cnf_file, strategy=["minimal"], n_components=0.95, motion_model="6params")

## Imputation with mean for the csf_derivatives and white_matter_derivatives NaNs for first row
confounds.fillna(confounds.mean(), inplace=True)

# cnf = confounds.to_numpy()
# assert (np.sum(np.isnan(cnf))==0),"Confounds contain NaNs!"
# assert (np.sum(np.isinf(cnf))==0),"Confounds contain infs!"

masker = NiftiMasker(mask_img= PATH + mask_file, standardize=True)
series = masker.fit_transform(PATH + bold_file, confounds = confounds.to_numpy())

(num_frames, num_channels) = series.shape

print("series shape = ", series.shape)

print("# frames = ", num_frames)
print("# voxels = ", num_channels)

series shape =  (202, 249518)
# frames =  202
# voxels =  249518


## Segmenting BOLD data time series into task blocks

In [0]:
# Get the file name for saving the task block series

def get_series_name(item):
  task = '2b' if '2' in item['block_type'] else '0b'
  stim = item['stim_type'].lower()
  return "{}_{}".format(task,stim)

In [0]:
# Extract and save the task block series

def extract_and_save_block_data(series, blocks):
  for index, item in blocks.iterrows():
    series_name = get_series_name(item)
    block_data = series[item['start_index'] : item['end_index']]
    print(series_name," of shape : ",block_data.shape)
    np.save(series_name, block_data)

In [51]:
extract_and_save_block_data(series, blocks)

2b_body  of shape :  (19, 249518)
0b_face  of shape :  (18, 249518)
2b_tools  of shape :  (19, 249518)
0b_body  of shape :  (19, 249518)
0b_place  of shape :  (18, 249518)
2b_face  of shape :  (19, 249518)
0b_tools  of shape :  (19, 249518)
2b_place  of shape :  (19, 249518)
